In [ ]:
# coding-utf-8
from deep_qnetwork import DeepQNetwork
import random
import copy

NUM_IN   = 
NUM_HID1 = 100
NUM_HID2 = 100
NUM_HID3 = 100
NUM_OUT  = 
BATCH_SIZE = 200
EPOCH = 1000
GAMMA = 0.9
EPSIL = 1.0
NUM_LIMIT_RECORD = 1000
NUM_INIT_RECORD = 10000
ProccerUnit = "CPU"
MODEL_NAME = "./DQN_model/"
ACTION = []
RESULT_REWARD = []

def agent_action(env, action):
    next_state, reward, terminal, info = env.step(action)
    return next_state, reward, terminal, info

def deep_Q_learn(env, Agent):
    STEP = 200
    Agent.load_weight()
    for episode in range(1, EPOCH+1):
        now_state = env.reset()
        sum_reward = 0
        for step in range(1, STEP+1):
            # 方策に従って行動を選択
            action, _, action_list = Agent.policy_egreedy(now_state, Agent.Q_network)
            # 行動による次状態を観測
            next_state, reward, term, _ = agent_action(env, [ACTION[action]])
            # ゴールの設定
            terminal, reward = (True, 1.0) if term else (False, reward)
    
            sum_reward += reward
            Agent.stock_record(now_state, action, next_state, reward, terminal, action_list)
                
            # 状態更新
            now_state = copy.deepcopy(next_state)
            # env.render()
            
            if terminal:
                break
            
            # Experience_Replayによるバッチ学習
            if len(Agent.record) > NUM_LIMIT_RECORD:
                Agent.init_record()
                if step % 20 == 0:
                    Agent.experience_replay_ver2()
                    Agent.EPSIL -= EPSIL / EPOCH
                if step % 40 == 0:
                    Agent.update_target_network()               
                    Agent.save_weight()
                
        print(episode, sum_reward, Agent.EPSIL)
        RESULT_REWARD.append(sum_reward)
    gragh(RESULT_REWARD, len(RESULT_REWARD))
                
def run():
    import gym
    env = gym.make("Pendulum-v0")
    Agent = DeepQNetwork(NUM_IN, NUM_HID1, NUM_HID2, NUM_HID3, NUM_OUT,
                         ProccerUnit, EPSIL, GAMMA, BATCH_SIZE, MODEL_NAME)
    deep_Q_learn(env, Agent)

In [ ]:
# coding:utf-8
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def gragh(result, epoch):    
    x = np.arange(0, epoch, 1)
    left = np.array(x)
    height = np.array(result)
    plt.plot(left, height)

In [ ]:
run()